In [ ]:
import os
import time
from dotenv import load_dotenv
import pandas as pd
import mltable
import jason

from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.entities import AmlCompute

from azure.ai.ml import load_component
from azure.ai.ml import dsl, Input

from azure.ai.ml.entities import PipelineJob
from IPython.core.display import HTML
from IPython.display import display

In [ ]:
load_dotenv()
SUBSCRIPTION_ID = os.environ.get("SUBSCRIPTION_ID")
RESOURCE_GROUP = os.environ.get("RESOURCE_GROUP")
AML_WORKSPACE_NAME = os.environ.get("AML_WORKSPACE_NAME")
print("AML_WORKSPACE_NAME:", AML_WORKSPACE_NAME)

credential = DefaultAzureCredential()
ml_client = MLClient(credential=credential, subscription_id=SUBSCRIPTION_ID, resource_group_name=RESOURCE_GROUP, workspace_name=AML_WORKSPACE_NAME, )

RAI registry

In [ ]:
# Get handle to azureml registry for the RAI built in components
version_string = "1"
compute_name = "rai-cluster"

registry_name = "azureml"
ml_client_registry = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION_ID,
    resource_group_name=RESOURCE_GROUP,
    registry_name=registry_name,
)
print(ml_client_registry)

In [ ]:
label = "latest"

rai_constructor_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_insight_constructor", label=label
)

# We get latest version and use the same version for all components
version = rai_constructor_component.version
print("The current version of RAI built-in components is: " + version)

rai_explanation_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_explanation", version=version
)

rai_causal_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_causal", version=version
)

rai_counterfactual_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_counterfactual", version=version
)

rai_erroranalysis_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_erroranalysis", version=version
)

rai_gather_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_insight_gather", version=version
)

rai_scorecard_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_score_card", version=version
)

In [ ]:
model_name_suffix = int(time.time())
model_name = "rai_housing_classifier"

expected_model_id = f"{model_name}_{model_name_suffix}:1"
azureml_model_id = f"azureml:{expected_model_id}"

In [ ]:
#train_model_component = load_component(source="../components/GradientBoostingClassifier/GradientBoostingClassifier.yaml")
train_model_component = load_component(source="../components/lgbm/lgbm.yaml")

In [ ]:
import json
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

target_feature = "Sold_HigherThan_Median"
categorical_features = []

classes_in_target = json.dumps(["Less than median", "More than median"])
treatment_features = json.dumps(
    ["OverallCond", "OverallQual", "Fireplaces", "GarageCars", "ScreenPorch"]
)

housing_train_pq = Input(
    type="mltable",
    path=f"azureml:housing_train_pq:1",
    mode="download",
)
housing_test_pq = Input(
    type="mltable",
    path=f"azureml:housing_test_pq:1",
    mode="download",
)


@dsl.pipeline(
    compute=compute_name,
    description="Register Model for RAI Housing example",
    experiment_name=f"RAI_Housing_Example_Model_Training_{model_name_suffix}",
)
def rai_demo_pipeline(target_column_name, training_data, test_data):
    trained_model = train_model_component(
        target_column_name=target_column_name,
        training_data=training_data,
        test_data=test_data,
        #exclude_features="NA",
        model_base_name=f"{model_name}_{model_name_suffix}"
    )
    trained_model.set_limits(timeout=3600)

    # Initiate the RAIInsights
    create_rai_job = rai_constructor_component(
        title="RAI Dashboard Example",
        #title = trained_model.outputs.model_output,
        task_type="classification",
        model_info=expected_model_id,
        #model_input=Input(type=AssetTypes.MLFLOW_MODEL, path=azureml_model_id),
        model_input=trained_model.outputs.model_output,
        train_dataset=training_data,
        test_dataset=test_data,
        target_column_name=target_column_name,
        categorical_column_names=json.dumps(categorical_features),
        classes=classes_in_target,
        # If your model has extra dependencies, and your Responsible AI job failed to
        # load mlflow model with ValueError, try set use_model_dependency to True.
        # If you have further questions, contact askamlrai@microsoft.com
        use_model_dependency=True,
    )
    create_rai_job.set_limits(timeout=7200)

    # Add error analysis
    erroranalysis_job = rai_erroranalysis_component(
        rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,
    )
    erroranalysis_job.set_limits(timeout=7200)

    
    # Combine everything
    rai_gather_job = rai_gather_component(
        constructor=create_rai_job.outputs.rai_insights_dashboard,
        #insight_1=explain_job.outputs.explanation,
        #insight_2=causal_job.outputs.causal,
        #insight_3=counterfactual_job.outputs.counterfactual,
        insight_4=erroranalysis_job.outputs.error_analysis,
    )
    rai_gather_job.set_limits(timeout=7200)

    rai_gather_job.outputs.dashboard.mode = "upload"
    rai_gather_job.outputs.ux_json.mode = "upload"

    """
    #return {"dashboard": rai_gather_job.outputs.dashboard,"ux_json": rai_gather_job.outputs.ux_json,}
    """
    return {}

rai_demo_pipeline_job = rai_demo_pipeline(target_feature, housing_train_pq, housing_test_pq)

In [ ]:
created_job = ml_client.jobs.create_or_update(rai_demo_pipeline_job)